# UPPP 135 Week 3: Geovisualization

<a target="_blank" href="https://colab.research.google.com/github/knaaptime/uppp135-winter26-assn/week3/geovisualization.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import pandas as pd
import geopandas as gpd
import mapclassify as mc
from fsspec import filesystem

In [ ]:
fs = filesystem('https')

tract_path="https://github.com/oturns/example_datasets/raw/refs/heads/main/acs/ca_tracts_2021.pq"
tracts = gpd.read_parquet(tract_path, filesystem=fs)

In [ ]:
oc_tracts = tracts[tracts.geoid.str.startswith('06059')]

The variable `p_edu_college_greater` contains the share of population over 25

In [ ]:
oc_tracts['p_edu_college_greater']

In [ ]:
oc_tracts['p_edu_college_greater'].describe()

In [ ]:
oc_tracts['p_edu_college_greater'].hist()

if we want to visualize this variable as a choropleth map, then we can use the `plot` method of the `tracts` geodataframe and pass the name of the column as the first argument of the function

In [ ]:
oc_tracts.plot('p_edu_college_greater')

what does this show?

In this case, we have just used the *default* plotting options to create the map. But we have some decisions to make

1. how do we assign values to colors?
2. which colors do we choose to represent variation?

The process of assigning colors to values is called *map classification* and we can use the `mapclassify` package to choose from different options. To see how these work, lets first look at how different classifiers break values into bins. To avoid typing to much (and because `mapclassify` doesnt like NaN values, lets assign the column to a new variable

In [ ]:
edu = oc_tracts["p_edu_college_greater"].dropna()

## Classification Schemes

these are all the classification schemes that are available to us

In [ ]:
mc.CLASSIFIERS

In [ ]:
equal_interval = mc.classify(edu, scheme='equal interval')

that gives an error because we didnt use the correct name. When a python package is written well, it gives an informative error message so we can try to figure out what we did wrong. In this case, the names dont have spaces

In [ ]:
equal_interval = mc.classify(edu, scheme='equalinterval')

In [ ]:
equal_interval.plot_legendgram(legend_size=("100%", "100%"))

unsurprisingly, this breaks the distribution into five equally-sized intervals, regardless of how many observations fall into each interval. If we wanted a different number, like 10 classes, we can add an argument to the `classify` function

In [ ]:
equal_interval10 = mc.classify(edu, scheme='equalinterval', k=10)

In [ ]:
equal_interval10.plot_legendgram(legend_size=('100%', '100%'))

to see where these breaks lie, we can look at other attributes of the mapclassify object

In [ ]:
equal_interval

to use these schemes in a map, we pass the `scheme` argument to the `plot` method of a dataframe

In [ ]:
oc_tracts.plot('p_edu_college_greater', scheme='equalinterval')

In [ ]:
oc_tracts.plot('p_edu_college_greater', scheme='equalinterval', k=10)

the `quantiles` classifier breaks the distribution into groups with equal numbers of observations (regardless of the space between bins)

In [ ]:
quantiles = mc.classify(edu, scheme='quantiles')

In [ ]:
quantiles

In [ ]:
quantiles.plot_legendgram(legend_size=("100%", "100%"))

to show the breaks and colors in the map, we add the argument `legend=True` to the plot method

In [ ]:
oc_tracts.plot('p_edu_college_greater', scheme='quantiles', legend=True)

the underlying data *are the same*, yet this map gives a different impression

In [ ]:
# how would we do an 8-class map?

the "jenks" classifiers assign values into bins that are most similar

In [ ]:
fisher_jenks = mc.classify(edu, scheme='fisherjenks')
fisher_jenks

In [ ]:
fisher_jenks.plot_legendgram(legend_size=("100%", "100%"))

In [ ]:
oc_tracts.plot('p_edu_college_greater', scheme='fisher_jenks')

when we call the `plot` function, it gives back a map. So far, we have just been displaying that map directly. But it can also be useful to save it into a variable. For instance if we want to plot the map and the histogram in the same figure.

(since 'plot' returns a `matplotlib.Axes` object, it's common to use the variable "ax")

In [ ]:
ax = oc_tracts.plot('p_edu_college_greater', scheme='fisherjenks', legend=True)
fisher_jenks.plot_legendgram(ax=ax)

the keyword `alpha` is used to change the transparency of the map layers. To add context to the map, we can use a package called `contextily` by plotting a basemap on the bottom and making the polygons partially transparent to see the basemap underneath

In [ ]:
import contextily as ctx

we can also make the map bigger using the `figsize` argument

In [ ]:
ax = oc_tracts.plot('p_edu_college_greater', scheme='fisherjenks', legend=True, alpha=0.6, figsize=(7,7))
ctx.add_basemap(ax=ax, source=ctx.providers.CartoDB.Positron, crs=oc_tracts.crs)
fisher_jenks.plot_legendgram(ax=ax)

map classification schemes also work with the `explore` method for creating interactive webmaps

In [ ]:
oc_tracts.explore('p_edu_college_greater', scheme='fisherjenks',tooltip='p_edu_college_greater')


it's a good idea to use a grayscale or dark basemap when making choropleth maps, otherwise the colors can cause confusion

In [ ]:
oc_tracts.explore('p_edu_college_greater', scheme='fisherjenks', tiles='cartodb positron',tooltip='p_edu_college_greater')

## Colormaps

you can also change the impression of a map by styling it differently, e.g. using a classifier to assign a different range of colors. In geopandas plotting, you can use any of the [matplotlib colormaps](https://matplotlib.org/stable/users/explain/colors/colormaps.html). This raises a few more things to consider

1. does the color gradient match the data gradient? (does the change between each of the colors match the change between each of the sets of values?)
2. do the values go light-to-dark, dark-to-light, or should they be centered on some central value?
3. are the colors equally accessible to everyone? 

the default colormap (yellow-green-blue) is called "viridis". It has 'cousins' called `plasma` and `magma` and we can change the colormap using the `cmap` keyword

In [ ]:
oc_tracts.explore('p_edu_college_greater', scheme='fisherjenks', cmap='plasma', tiles='cartodb positron',tooltip='p_edu_college_greater')

In [ ]:
oc_tracts.explore('p_edu_college_greater', scheme='fisherjenks', cmap='magma', tiles='cartodb positron',tooltip='p_edu_college_greater')

these all have the same data and use the same classification scheme, but generate different maps.

You can also reverse any colormap by adding the `_r` suffix (for reverse)

In [ ]:
# use reverse magma
oc_tracts.explore('p_edu_college_greater', scheme='fisherjenks', cmap='magma_r', tiles='cartodb positron',tooltip='p_edu_college_greater')

all of these maps step through multiple colors. But sometimes it's better to step through a range of *one* color because its easier to interpret the distance between two bins

In [ ]:
for color in ['Reds', 'Blues', 'Greens']:
    oc_tracts.plot('p_edu_college_greater', scheme='fisherjenks', cmap=color)

there are some combinations you should avoid. For instance, this used to be common

In [ ]:
oc_tracts.explore('p_edu_college_greater', scheme='fisherjenks', cmap='RdYlGn', tiles='cartodb positron',tooltip='p_edu_college_greater')

you shouldn't use a map like this because 
1. its ugly and
2. red-green colorblindness is the [most common form of color deficiency](https://www.colourblindawareness.org/colour-blindness/types-of-colour-blindness/)

But sometimes it is useful to pick a colormap with "poles". In such cases it's important to pair with the proper classification scheme

In [ ]:
oc_tracts.explore('p_edu_college_greater', scheme='quantiles', cmap='RdBu_r', tiles='cartodb positron',tooltip='p_edu_college_greater')

in this map, we've chosen 5 quantiles and a colormap that ranges from blue to red with white in the middle. This is a good match because blue represents the low end of our value range (blue usually means low or "cold") and red representing the high range (usually meaning high or "hot"). The values in the middle quintile near the 50th percentile are shown in white.

in short, red places are higher than the median, blue places are lower than the median